In [6]:
# Creates a database, clears if exists and fills with training data
import sqlite3
import codecs
import random

# Connect to database (using sqlite3 lib built into python)
conn = sqlite3.connect( 'comp3208.db')

# Read in training data file
readHandle = codecs.open( 'comp3208-train-small.csv', 'r', 'utf-8', errors = 'replace' )
listLines = readHandle.readlines()
readHandle.close()

# Creates a table for training data, deletes all data
c = conn.cursor()
c.execute( 'CREATE TABLE IF NOT EXISTS training_table (UserID INT, ItemID INT, Rating FLOAT, PredRating FLOAT)' )
conn.commit()
c.execute( 'DELETE FROM training_table' )
conn.commit()
# Inserts data from csv file into training data table (userid, itemid, rating, timestamp - currently not included)
for strLine in listLines :
    if len(strLine.strip()) > 0 :
        listParts = strLine.strip().split(',')
        if len(listParts) == 4 :
            # Insert training set into table with an initial predicted rating of 0
            c.execute( 'INSERT INTO training_table VALUES (?,?,?,?)', (listParts[0], listParts[1], listParts[2], 0) )
        else :
            raise Exception( 'failed to parse csv : ' + repr(listParts) )
conn.commit()

# Creates an index of the user ID and item ID
c.execute( 'CREATE INDEX IF NOT EXISTS training_table_index on training_table (UserID, ItemID)' )
conn.commit()

In [7]:
# Inserts testing data into a table in database

# Reads in data 
readHandle = codecs.open( 'comp3208-test-small.csv', 'r', 'utf-8', errors = 'replace' )
listLines = readHandle.readlines()
readHandle.close()

# Create and clear table for testing data
c = conn.cursor()
c.execute( 'CREATE TABLE IF NOT EXISTS testing_table (UserID INT, ItemID INT, Rating FLOAT, PredRating FLOAT)' )
conn.commit()
c.execute( 'DELETE FROM testing_table' )
conn.commit()
# Inserts data from csv file into testing data table (userid, itemid, timestamp - currently not used)
for strLine in listLines :
    if len(strLine.strip()) > 0 :
        listParts = strLine.strip().split(',')
        if len(listParts) == 3 :
            # Insert training set into table with a rating and predicted rating of 0
            c.execute( 'INSERT INTO testing_table VALUES (?,?,?,?)', (listParts[0], listParts[1], 0, 0) )
        else :
            raise Exception( 'failed to parse csv : ' + repr(listParts) )
conn.commit()

# Creates an index of the user ID and item ID
c.execute( 'CREATE INDEX IF NOT EXISTS testing_table_index on testing_table (UserID, ItemID)' )
conn.commit()

In [ ]:
# Compute MSE of SQL data
c.execute('SELECT AVG(ABS(Rating-PredRating)) FROM training_table WHERE PredRating IS NOT NULL')
row = c.fetchone()
nMSE = float( row[0] )

print('MSE for random prediction: ' + str(nMSE) )

# Compute MSE against a fixed average rating using SQL
c.execute('SELECT AVG(ABS(Rating-3.53)) FROM training_table WHERE PredRating IS NOT NULL')
row = c.fetchone()
nMSE = float( row[0] )

print('MSE for user average of 3.53 prediction: ' + str(nMSE) )

# Close database connection
c.close()
conn.close()


MSE for random prediction: 3.5032546495524066


In [ ]:
c.execute('SELECT * FROM training_table')
print(c.fetchone())